In [74]:
FIXED_IP = [2, 6, 3, 1, 4, 8, 5, 7]
FIXED_EP = [4, 1, 2, 3, 2, 3, 4, 1]
FIXED_IP_INVERSE = [4, 1, 3, 5, 7, 2, 8, 6]
FIXED_P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
FIXED_P8 = [6, 3, 7, 4, 8, 5, 10, 9]
FIXED_P4 = [2, 4, 3, 1]

S0 = [[1, 0, 3, 2],
      [3, 2, 1, 0],
      [0, 2, 1, 3],
      [3, 1, 3, 2]]

S1 = [[0, 1, 2, 3],
      [2, 0, 1, 3],
      [3, 0, 1, 0],
      [2, 1, 0, 3]]

def permutate(original, fixed_key):
    new = ''
    for i in fixed_key:
        new += original[i - 1]
    return new


def left_half(bits):
    return bits[:len(bits)//2]


def right_half(bits):
    return bits[len(bits)//2:]


def shift(bits):
    rotated_left_half = left_half(bits)[1:] + left_half(bits)[0]
    rotated_right_half = right_half(bits)[1:] + right_half(bits)[0]
    return rotated_left_half + rotated_right_half


def key1():
    return permutate(shift(permutate(KEY, FIXED_P10)), FIXED_P8)


def key2():
    return permutate(shift(shift(shift(permutate(KEY, FIXED_P10)))), FIXED_P8)


def xor(bits, key):
    new = ''
    for bit, key_bit in zip(bits, key):
        new += str(((int(bit) + int(key_bit)) % 2))
    return new


def lookup_in_sbox(bits, sbox):
    row = int(bits[0] + bits[3], 2)
    col = int(bits[1] + bits[2], 2)
    return '{0:02b}'.format(sbox[row][col])


def f_k(bits, key):
    L = left_half(bits)
    R = right_half(bits)
    bits = permutate(R, FIXED_EP)
    bits = xor(bits, key)
    bits = lookup_in_sbox(left_half(bits), S0) + lookup_in_sbox(right_half(bits), S1)
    bits = permutate(bits, FIXED_P4)
    return xor(bits, L)


def encrypt(plain_text):
    bits = permutate(plain_text, FIXED_IP)
    temp = f_k(bits, key1())
    bits = right_half(bits) + temp
    bits = f_k(bits, key2())
    res = permutate(bits + temp, FIXED_IP_INVERSE)
    return res


def decrypt(cipher_text):
    bits = permutate(cipher_text, FIXED_IP)
    temp = f_k(bits, key2())
    bits = right_half(bits) + temp
    bits = f_k(bits, key1())
    res = permutate(bits + temp, FIXED_IP_INVERSE)
    #print (res)
    return res

def encrypt_string(text):
    res = []
    view = ''
    lk = ''
    for i in text:
        #res.append(chr(int(encrypt(format(ord(i), '08b')),2)))
        lk = encrypt(format(ord(i), '08b'))
        res.append(lk)
        view += (chr(int(lk,2)))
    print(view)
    return res

def decrypt_string(aux):
    lk = []
    for i in aux:
        lk.append(chr(int(decrypt(i),2)))
    return "".join(lk)

def string_to_list(string):
    box = ''
    for i in range(0, len(string)) :
        if i%8 == 0 and i > 0:
            box += 'x'
        box += string[i]
    return (box.split('x'))

def SDES(key, text): 
    global KEY 
    KEY = key
    tmp = (encrypt_string(text))
    return "".join(tmp)

def SDES_Decode(key, text):
    global KEY 
    KEY = key
    tmp = string_to_list(text)
    return decrypt_string(tmp)


In [73]:
(SDES("1010000010","Technology" ))

/©


'00100010111110001110100001001100010110100010111100001101001011111010100100000110'

In [71]:
SDES_Decode("1010000010",'00100010111110001110100001001100010110100010111100001101001011111010100100000110')

'Technology'

In [9]:
#example
#KEY= '1100100001'
#CODE = encrypt_string("Technology is a way of organizing the universe so that man doesn't have to experience it.")
#DECODE = decrypt_string(CODE)

encrypted:  100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101100011011000110110001101


In [109]:
#res = '110001011101101110110101010001111100011111111110110001101111111011010000110100110010100101001100000101100010100101011110001010010111001101011110110100110010100111111110101010010010100111111110001001101101000001011110110001110100110011001001010011001100011111010000001010010110100101000111110110110010100101001010110001110100110010100111110110110010011000010110110110110010100100010110111111100010100101101001010001110101111001101001001010010000110101011110110001110010100111111100111111101101101100010110110001110100010001101001001010010100011101011110101001111101101100101001011010011111111000101001110110110000011100101000110110110010011001001100110110111100011110110101110110110010100101001100011010010001011111111010'
#print("Expected:", res)

In [110]:
#def main():
#    print('enter msg :')
#    word = input()
#    _in = encrypt_string(word)
#    print("encrypted: ",   show_code(_in))
#    _out = decrypt_string(_in)
#    print("decrypted: ", _out)
#    
#if __name__ == "__main__":
#
#    main()

enter msg :
teste de chave
encrypted:  11010011101101110110110100111011011101001111111001101101110100110110101100011110111101010011111011011
decrypted:  teste de chave
